## Point Classifier
Determine wheter a point is above or below an arbitrary line in 2D space<br>
Last update: 1/1/24

In [1]:
import cupy as np
from importlib import reload
from sandbox import activations, costs, initializers, layers, model, optimizers, utils

In [2]:
line = lambda x: x**2 - 2
mean = 0
sd = 5
num_train = 100
num_test = 100

# Create training and test input data - Points in 2D space with given mean and sd
train_x = np.random.normal(mean, sd, (num_train, 2))
test_x = np.random.normal(mean, sd, (num_test, 2))

# Create labels by determining whether the input is above or below the line
train_y = np.where(train_x[:, 1] > line(train_x[:, 0]), 1, 0).reshape(-1, 1)
test_y = np.where(test_x[:, 1] > line(test_x[:, 0]), 1, 0).reshape(-1, 1)

In [13]:
reload(model)

# Configure all modules to run on Cuda
utils.configure_cupy()

# Create model
point = model.Model()
point.add(layers.Dense(units=8, activation=activations.ReLU()))
point.add(layers.Dense(units=8, activation=activations.ReLU()))
point.add(layers.Dense(units=1, activation=activations.Sigmoid()))

point.configure(
    input_size=train_x.shape[1],
    cost_type=costs.BinaryCrossentropy(),
    optimizer=optimizers.Adam()
)

# Train model
point.train(train_x, train_y, epochs=80, batch_size=64, verbose=True)

Cost on epoch 8: 0.28703
Cost on epoch 16: 0.21849
Cost on epoch 24: 0.18554
Cost on epoch 32: 0.16301
Cost on epoch 40: 0.14441
Cost on epoch 48: 0.13074
Cost on epoch 56: 0.1201
Cost on epoch 64: 0.11113
Cost on epoch 72: 0.10336
Cost on epoch 80: 0.0965


In [14]:
# Evaluate model
pred_train = point.predict(train_x)
print('Training Accuracy:', utils.evaluate(pred_train, train_y))
pred_test = point.predict(test_x)
print('Testing Accuracy:', utils.evaluate(pred_test, test_y))

Training Accuracy: 0.96
Testing Accuracy: 0.97
